In [1]:
import tensorflow as tf

In [2]:
# hello tensorflow
a = tf.constant(3, name="const1") # 定数a
b = tf.Variable(0, name="val1") # 変数b
add = tf.add(a, b) # aとbの加算

# 変数bに加算結果をアサイン（割り当て）
assign = tf.assign(b, add)
c = tf.placeholder(tf.int32, name="input") # 入力c

# アサイン結果とcを乗算
mul = tf.multiply(assign, c)

# 変数の初期化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # 初期化を実行
    sess.run(init)
    for i in range(3):
        # 乗算を行うまでのループを実行
        print(sess.run(mul, feed_dict={c:3}))

9
18
27


In [3]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(3):
        print(sess.run(add))

3
3
3


In [4]:
# mnistのダウンロード
from tensorflow.examples.tutorials.mnist import input_data

# データを格納したオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [5]:
# 訓練用の入力データ，正解データをミニバッチ数を指定して取得
train_images, train_labels = mnist.train.next_batch(50)

# テスト用の全画像データを取得
test_images = mnist.test.images
# テスト用の全正解データを取得
test_labels = mnist.test.labels

In [6]:
test_images.shape

(10000, 784)

In [7]:
# モデルの定義

# 入力データを定義
x = tf.placeholder(tf.float32, [None, 784])

# 入力画像をログに出力
img = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image("input_data", img, 10)
 
# 入力層から中間層
with tf.name_scope("hidden"):
    w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
    b_1 = tf.Variable(tf.zeros([64]), name="b1")
    h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)
    # 中間層の重みの分布をログ出力
    tf.summary.histogram("w_1", w_1)

# 中間層から出力層
with tf.name_scope("output"):
    w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
    b_2 = tf.Variable(tf.zeros([10]), name="b2")
    out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

y = tf.placeholder(tf.float32, [None, 10])

# 誤差関数
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y - out))
    # 誤差をログ出力
    tf.summary.scalar("loss", loss)

In [8]:
# 訓練（分散処理だと別の書き方をするらしい...）
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 評価
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # 精度をログ出力
    tf.summary.scalar("accuracy", accuracy)

# 一部のログをマージ
#merge = tf.summary.merge([a, b, c])

# 全てのログをマージ
summary_op = tf.summary.merge_all()

# 初期化
init = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter("logs", sess.graph)
    
    # テストデータをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels

    for i in range(1000):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images, y:train_labels})
    
        if step % 10 == 0:
            summary_str = sess.run(summary_op, feed_dict={x:test_images, y:test_labels})
            summary_writer.add_summary(summary_str, step)
            acc_val = sess.run(accuracy, feed_dict={x:test_images, y:test_labels})
            print("step %d: accuracy = %.2f" % (step, acc_val))

step 10: accuracy = 0.08
step 20: accuracy = 0.10
step 30: accuracy = 0.12
step 40: accuracy = 0.13
step 50: accuracy = 0.16
step 60: accuracy = 0.21
step 70: accuracy = 0.26
step 80: accuracy = 0.31
step 90: accuracy = 0.34
step 100: accuracy = 0.38
step 110: accuracy = 0.41
step 120: accuracy = 0.41
step 130: accuracy = 0.42
step 140: accuracy = 0.45
step 150: accuracy = 0.46
step 160: accuracy = 0.48
step 170: accuracy = 0.49
step 180: accuracy = 0.50
step 190: accuracy = 0.52
step 200: accuracy = 0.52
step 210: accuracy = 0.54
step 220: accuracy = 0.56
step 230: accuracy = 0.56
step 240: accuracy = 0.58
step 250: accuracy = 0.59
step 260: accuracy = 0.61
step 270: accuracy = 0.63
step 280: accuracy = 0.66
step 290: accuracy = 0.70
step 300: accuracy = 0.71
step 310: accuracy = 0.73
step 320: accuracy = 0.75
step 330: accuracy = 0.75
step 340: accuracy = 0.78
step 350: accuracy = 0.78
step 360: accuracy = 0.79
step 370: accuracy = 0.81
step 380: accuracy = 0.81
step 390: accuracy = 

In [ ]:
!tensorboard --logdir ./logs

# ここからCNN

In [10]:
# 入出力データを定義
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
# （バッチサイズ、高さ、横幅、チャンネル数）に変更
img = tf.reshape(x, [-1, 28, 28, 1])

# 畳み込み層1
f1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
conv1 = tf.nn.conv2d(img, f1, strides=[1, 1, 1, 1], padding="SAME")
b1 = tf.Variable(tf.constant(0.1, shape=[32]))
h_conv1 = tf.nn.relu(conv1 + b1)

# プーリング層1
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 畳み込み層2
f2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
conv2 = tf.nn.conv2d(h_pool1, f2, strides=[1, 1, 1, 1], padding='SAME')
b2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(conv2 + b2)

# プーリング層2
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 畳み込まれているものをフラットな形に変換
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# 全結合層
w_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# 出力層
w_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
out = tf.nn.softmax(tf.matmul(h_fc1, w_fc2) + b_fc2)

y = tf.placeholder(tf.float32, [None, 10])

# 誤差関数
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 1e-5), axis=[1]))

In [11]:
# 訓練
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# 評価
correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

"""実行部分"""

saver = tf.train.Saver() # モデルの保存

with tf.Session() as sess:
    sess.run(init)
    # テストデータをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for step in range(1000):
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images, y:train_labels})
        
        if step % 100 == 0:
            acc_val = sess.run(accuracy, feed_dict={x:test_images, y:test_labels})
            print("step %d: accuracy = %.2f" % (step, acc_val))

step 0: accuracy = 0.10
step 100: accuracy = 0.88
step 200: accuracy = 0.91
step 300: accuracy = 0.93
step 400: accuracy = 0.94
step 500: accuracy = 0.95
step 600: accuracy = 0.95
step 700: accuracy = 0.96
step 800: accuracy = 0.96
step 900: accuracy = 0.96


In [ ]:
# モデルの保存
